In [1]:
#!pip install --upgrade tensorflow==1.15

In [2]:
#Import Library
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

In [3]:
# Input Data
cols = df.columns
features = df[cols[0:4]].values 
label = df[['Crime Code']].values.reshape(-1,1)

NameError: ignored

In [ ]:
# Split Data
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.2) 

In [ ]:
# Preprocessing Data
scaler = StandardScaler().fit(x_train) 
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
# Preprocessing Label
encoder = OneHotEncoder().fit(y_train)
y_train = encoder.transform(y_train).toarray()
y_test = encoder.transform(y_test).toarray()

In [ ]:
# Convert Data to float 32
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
y_train, y_test = y_train.astype('float32'), y_test.astype('float32')

In [ ]:
epochs = 5000
learning_rate = 0.01
n_features = x_train[0].shape[0]
n_classes = y_train.shape[1]

In [ ]:
# Create a placeholder
input_tensor = tf.placeholder(tf.float32, name = 'input')
label_tensor = tf.placeholder(tf.float32, name = 'label')

In [ ]:
neurons = [n_features, 4 ,n_classes]
n_layers = len(neurons)


In [ ]:
parameters = {
    'W1' : tf.Variable(
        tf.random.normal([
            n_features, neurons[0]
        ]),
        tf.float32
    ),
    'B1' : tf.Variable(
        tf.random.normal([1,  neurons[0]]),
        tf.float32
    ),
    'W2' : tf.Variable(
        tf.random.normal([
            neurons[0], neurons[1]
        ]),
        tf.float32
    ),
    'B2' : tf.Variable(
        tf.random.normal([1,  neurons[1]]),
        tf.float32
    ),
    'W3' : tf.Variable(
        tf.random.normal([
            neurons[1], neurons[2]
        ]),
        tf.float32
    ),
    'B3' : tf.Variable(
        tf.random.normal([1,  neurons[2]]),
        tf.float32
    )
}

In [ ]:
#Forward Pass
def forward_pass(input_tensor,parameter,n_layer=3): 
  a = input_tensor
  for i in range(n_layer):
    W = parameter[f'W{i+1}']
    B = parameter[f'B{i+1}']

    z = tf.matmul(a,W) + B 
    if i == n_layer-1:
      a = tf.nn.sigmoid(z)
    else:
      a = tf.nn.sigmoid(z)

  return a 

In [ ]:
def evaluate_accuracy(y, yhat):
  yield (0.5 * (yhat - y)**2).sum(axis=1).mean()
  yield accuracy_score(np.argmax(y, axis = 1), np.argmax(yhat, axis = 1))

In [ ]:
z = forward_pass()

In [ ]:
with tf.Session() as sess:
  logits_tensor = forward_pass(input_tensor, parameters)
  loss_tensor = tf.losses.mean_squared_error(label_tensor, logits_tensor)
  true_preds_tensor = tf.equal(tf.argmax(logits_tensor, axis=1), tf.argmax(label_tensor, axis=1))
  acc_tensor = tf.reduce_mean(tf.cast(true_preds_tensor, tf.float32))

  optimiser = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_tensor)
  sess.run(tf.global_variables_initializer())
  for i in range(epochs+1):
    sess.run(
        optimiser,
        feed_dict = {
            input_tensor: x_train,
            label_tensor: y_train
        }
    )

    #Validation
    if i%100 == 0:
      loss,acc = evaluate_accuracy(y_test,sess.run(
          logits_tensor,
          feed_dict = {
              input_tensor: x_test,
              label_tensor: y_test
          }
      ))
      print(f'Epoch : {i}')
      print(f'loss : {loss:.2f}')
print(f'Accuracy : {acc * 100:.2f}%')